In [1]:
# makes a two-catchment open book for testing of multiple communicators and multiple subdomain solves
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import numpy as np
from matplotlib import pyplot as plt

import workflow.mesh

In [3]:
import pymetis
import h5py


In [4]:
xl = np.arange(-510.0, -10.0, 50.0)
xr = -np.flipud(xl)

x = np.concatenate([xl, np.array([-10.0,10.0]), xr])
y = np.arange(-1000,1001,50)

print((len(x)-1)*(len(y)-1)*10)

8400


In [5]:
X,Y = np.meshgrid(x,y)

In [6]:
#plt.scatter(X.ravel(), Y.ravel())

In [7]:
dy = 0.02
dx = 0.05
def elevation(x,y):
    if x < -10:
        z0 = (-10-x)*dx
    elif x > 10:
        z0 = (x-10)*dx
    else:
        z0 = 0.

    return z0 + (y + 1000)*dy

Z = np.array([elevation(xx,yy) for (xx,yy) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)

#plt.scatter(X.ravel(), Y.ravel(), c=Z.ravel())

In [8]:
inds = np.zeros(X.shape, 'int')
inds.ravel()[:] = np.arange(len(X.ravel()))
print(inds.shape)

(41, 22)


In [9]:
cells = -1 * np.ones((X.shape[0]-1, X.shape[1]-1, 4),'i')
print(cells.shape)
cells[:,:,0] = inds[0:-1,0:-1]
cells[:,:,1] = inds[1:, 0:-1]
cells[:,:,2] = inds[1:, 1:]
cells[:,:,3] = inds[0:-1,1:]
cells = cells.reshape((-1, 4))



(40, 21, 4)


In [10]:
coords = np.array([X.ravel(), Y.ravel(), Z.ravel()]).transpose()
print(coords.shape)

(902, 3)


In [11]:
uc = list(range(0,int(len(cells)/2)))
dc = list(range(int(len(cells)/2),len(cells)))

upstream = workflow.mesh.LabeledSet('upstream catchment surface', 10000, "CELL", uc)
downstream = workflow.mesh.LabeledSet('downstream catchment surface', 10001, "CELL", dc)

In [12]:
conn = [[c for c in sq] for sq in cells]
#print(len(conn))
print(coords.shape[1])
m2 = workflow.mesh.Mesh2D(coords, conn, [upstream,downstream])

3


In [13]:
# split cores across subdomain
num_proc_total = 4

partition = {'upstream':upstream,
             'downstream':downstream}
coloring = -np.ones(m2.num_cells(), 'i')

# create reverse map from cell id to index in partition, set up data structures
partition_reverse_maps = dict()
partition_adjacencies = dict()
partition_core_counts = dict()

for partname, part in partition.items():
    my_map = dict()
    partition_reverse_maps[partname] = my_map
    for i, ent in enumerate(part.ent_ids):
        my_map[ent] = i

    partition_adjacencies[partname] = [list() for i in range(len(part.ent_ids))]
    partition_core_counts[partname] = int(np.round(num_proc_total * len(part.ent_ids) / m2.num_cells()))

# note, we would do better to sort these somehow and give/take cores
# to the ones furthest from their rounded values...
init_core_count = sum(my_cores for my_cores in partition_core_counts.values())
if (init_core_count < num_proc_total):
    cores_to_give = num_proc_total - init_core_count
    for partname, i in zip(partition_core_counts, range(cores_to_give)):
        partition_core_counts[partname] += 1
elif (init_core_count > num_proc_total):
    cores_to_take = init_core_count - num_proc_total
    for partname, i in zip(partition_core_counts, range(cores_to_take)):
        partition_core_counts[partname] -= 1

# create the adjacency structure of the partition's cells
for e in m2.edges():
    # find the cells on either side of this face
    e_cells = m2.edge_to_cells(*e)
    if len(e_cells) == 2:
        # interior face, connecting two cells -- is it a cut face?
        found = False
        for partname, map in partition_reverse_maps.items():
            if e_cells[0] in map and e_cells[1] in map:
                found = True
                break

        if found:
            # place the adjacency in both cell lists
            c1 = map[e_cells[0]]
            c2 = map[e_cells[1]]
            partition_adjacencies[partname][c1].append(c2)
            partition_adjacencies[partname][c2].append(c1)

# further partition each partition
init_color = 0
for partname, adjacency_info in partition_adjacencies.items():
    n_cuts, part_coloring = pymetis.part_graph(partition_core_counts[partname], adjacency=partition_adjacencies[partname])
    for id, color in enumerate(part_coloring):
        coloring[partition[partname].ent_ids[id]] = color + init_color
    init_color += partition_core_counts[partname]

print(coloring)
            


[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0
 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 3 3 3
 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2
 2 2 3 3 3 3 3 3 3 3 3 3 

In [14]:
# plot the centroids
#m2.plot(color=coloring)


In [15]:
m3 = workflow.mesh.Mesh3D.extruded_Mesh2D(m2, ['constant',], [2,], [10,], [100,])

In [17]:
# extrude the labeled sets and add cell-based labeled sets based on upstream and downstream labels.
upstream_cells = workflow.mesh.LabeledSet('upstream catchment', 10000, "CELL",
                                          [face_id*m3.ncells_tall + j for face_id in upstream.ent_ids for j in range(m3.ncells_tall)])
downstream_cells = workflow.mesh.LabeledSet('downstream catchment', 10001, "CELL",
                                            [face_id*m3.ncells_tall + j for face_id in downstream.ent_ids for j in range(m3.ncells_tall)])
m3.addLabeledSet(upstream_cells)
m3.addLabeledSet(downstream_cells)

print('upstream size:', len(upstream_cells.ent_ids))
print('downstream size:', len(downstream_cells.ent_ids))

# these must be written to ANOTHER file because we have no way of getting two colorings into an exodus file
with h5py.File('subdomain_coloring.h5', 'w') as fid:
    fid.create_dataset('upstream', data=np.array(upstream_cells.ent_ids, 'i')+1)
    fid.create_dataset('downstream', data=np.array(downstream_cells.ent_ids, 'i')+1)



upstream size: 4200
downstream size: 4200


In [17]:
m3.write_exodus('double_open_book.exo')


You are using exodus.py v 1.20.0 (seacas-py3), a python wrapper of some of the exodus library.

Copyright (c) 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020 National Technology &
Engineering Solutions of Sandia, LLC (NTESS).  Under the terms of
Contract DE-NA0003525 with NTESS, the U.S. Government retains certain
rights in this software.

Opening exodus file: double_open_book.exo
Closing exodus file: double_open_book.exo


In [18]:
# write a coloring file
extruded_old_coloring = [coloring[i] for i in range(m3.num_cols) for j in range(m3.ncells_tall)]
assert(all(np.array(extruded_old_coloring[0::m3.ncells_tall]) == np.array(coloring)))
new_coloring = -np.ones(m3.num_cells(), 'i')

# note the new coloring must be remapped as the exodus file was remapped
for c in range(m3.num_cells()):
    new_coloring[m3.old_to_new_elems[c]] = extruded_old_coloring[c]

import struct
with open('coloring.bin', 'wb') as fid:
    for c in new_coloring:
        fid.write(struct.pack('i', c))


# this test doesn't test the remapping, because there is only one material id...
assert(all(np.array(extruded_old_coloring) == np.array(new_coloring)))

# now go partition with
# ❯ mpiexec --oversubscribe -n 4 meshconvert --classify=1 --partition=1 --partition-method=3 double_open_book.exo 4/double_open_book.par
